# OLD pyg_dataset_sandbox
This was code from my first attempt at Cora, but I realized the way this was done it doesn't line up well with using 
DataLoaders/minibatching which I think I want to do

In [ ]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

# number of graphs in dataset
print(f'Dataset:')
print(f'{len(dataset)} total graphs')
print(f'{dataset.num_features:,} features')
print(f'{dataset.num_edge_features:,} edge features, {dataset.num_node_features:,} node features')
# [num nodes, num node features]
print(f'Shape is [num nodes, num features]: {dataset[0].x.shape}')

Dataset:
1 total graphs
1,433 features
0 edge features, 1,433 node features
Shape is [num nodes, num features]: torch.Size([2708, 1433])


In [ ]:
# move to GPU
data = dataset[0].to(device)
print(f'Before: {dataset[0].x.device}')
print(f'After: {data.x.device}')

Before: cpu
After: cuda:0


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)

        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

In [ ]:
data.x[data.train_mask].shape

torch.Size([140, 1433])

In [ ]:
data.y[data.train_mask]

tensor([3, 4, 4, 0, 3, 2, 0, 3, 3, 2, 0, 0, 4, 3, 3, 3, 2, 3, 1, 3, 5, 3, 4, 6,
        3, 3, 6, 3, 2, 4, 3, 6, 0, 4, 2, 0, 1, 5, 4, 4, 3, 6, 6, 4, 3, 3, 2, 5,
        3, 4, 5, 3, 0, 2, 1, 4, 6, 3, 2, 2, 0, 0, 0, 4, 2, 0, 4, 5, 2, 6, 5, 2,
        2, 2, 0, 4, 5, 6, 4, 0, 0, 0, 4, 2, 4, 1, 4, 6, 0, 4, 2, 4, 6, 6, 0, 0,
        6, 5, 0, 6, 0, 2, 1, 1, 1, 2, 6, 5, 6, 1, 2, 2, 1, 5, 5, 5, 6, 5, 6, 5,
        5, 1, 6, 6, 1, 5, 1, 6, 5, 5, 5, 1, 5, 1, 1, 1, 1, 1, 1, 1],
       device='cuda:0')

In [ ]:
# NOTE: ok, looks like this is a BAD EXAMPLE that doesn't work well for DataLoaders :P
# thanks Cora...
# ----------------------------------------
# train_dataset = data.x[data.train_mask]
# val_dataset = data.x[data.val_mask]
# test_dataset = data.x[data.test_mask]

# ----------------------------------------
# TODO: pick up using a different dataset...follow along with one of the actual PyG tutorials (colab)
# ----------------------------------------



In [ ]:
len(data.x)
train_size = int(0.5 * len(data.x))
test_size = int(0.3 * len(data.x))
val_size = len(data.x) - train_size - test_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(data, [train_size, val_size, test_size])

ValueError: Sum of input lengths does not equal the length of the input dataset!

In [ ]:
from torch_geometric.loader import DataLoader

model = GCN(dataset.num_node_features, hidden_channels=64, num_classes=dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

num_epochs = 5

model.train()
for epoch in range(num_epochs):
    print(f'Epoch {epoch} of {num_epochs}...')
    for batch_x, batch_y in train_loader:
        optimizer.zero_grad()
        out = model(batch_x)
        loss = F.nll_loss(out, batch_y)
        # loss = F.nll_loss(out[batch.train_mask], batch.y[batch.train_mask])
        loss.backward()
        optimizer.step()


Epoch 0 of 5...


ValueError: too many values to unpack (expected 2)